# Research Request - GTFS Digest: Add Rail and Ferry Operators. #1386
# Focus on Ferry here. 
Tiffany's comment:
If it's just a couple of rail, (Amtrak, Metrolink) and a handful of ferry operators, it's worth digging into the why they dropped off, and start by looking for their rows in the 4 schedule tables: trips, shapes, stops, stop_times, and then look for it in a vp table.

* I think the ferry operators and Metrolink are already associated to a district. Even Amtrak might be? But if Amtrak isn't, you can create a separate "district = Amtrak" the merged df so it always has a tab for itself. Amtrak plots for the entire country!
* District 4: San Francisco Bay Area Rapid Transit (BART), City and County of San Francisco (Muni)
* District 7: Los Angeles County Metropolitan Transportation Authority (LA Metro)
* District 11: San Diego Metropolitan Transit System

Amanda
* Ferry operator: Bay Area WETA, City of Alameda, and Golden Gate Bridge, Highway and Transportation District show up. All 3 are vp_only so they were filtered out -> incorporate them in? 
* The only ferry operator missing is Santa Cruz Harbor. 
* Amtrak is in District 3 but it has schedule_only data, which isn't true? 

* Here's a list of ferry operators in California from Evan's comment [here](https://github.com/cal-itp/data-analyses/issues/1357):
    
    * City of Alameda
    * Golden Gate
    * SF WETA
    * Santa Cruz Harbor
* **Goal**: all operators that are vp_only should also have schedule data. It is not possible to have realtime data without scheduled data. 

In [1]:
import _section1_utils as section1
import _section2_utils as section2
import geopandas as gpd
import merge_data
import merge_operator_data
import numpy as np
import pandas as pd
import geopandas as gpd
from segment_speed_utils import gtfs_schedule_wrangling, helpers
from segment_speed_utils.project_vars import COMPILED_CACHED_VIEWS, PROJECT_CRS
from shared_utils import catalog_utils, portfolio_utils, rt_dates
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
analysis_date_list = [rt_dates.DATES["feb2025"]]

In [4]:
analysis_date = rt_dates.DATES["feb2025"]

In [5]:
schd_vp_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.route_schedule_vp}.parquet"

In [6]:
EXPORT = GTFS_DATA_DICT.schedule_tables.route_typologies

In [7]:
route_typologies = pd.read_parquet(f"{SCHED_GCS}{EXPORT}_{analysis_date}.parquet")

## Look at `helpers.remove_shapes_outside_ca`

In [8]:
us_states = gpd.read_file(
        "https://services.arcgis.com/ue9rwulIoeLEI9bj/"
        "arcgis/rest/services/US_StateBoundaries/FeatureServer/0/"
        "query?outFields=*&where=1%3D1&f=geojson"
    )
    
border_states = ["CA", "NV", "AZ", "OR"]
    

    # Filter to California
ca = us_states.query(
        'STATE_ABBR in @border_states'
    ).dissolve()[["geometry"]]

In [9]:
ca.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [10]:
ca2 = ca.buffer(100)

/tmp/ipykernel_643/643034411.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ca2 = ca.buffer(100)


### This shape doesn't keep parts of the Bay Area. 

In [11]:
ca.explore()

## Look at operators in `digest/schedule_vp_metrics` without any filters to see if ferry and rail operators are in here.
* Ferry operators except Bay Area Water Emergency Services (which isn't even a ferry?) aren't here.

In [12]:
schd_vp_df = pd.read_parquet(
    schd_vp_url,
    columns=[
        "schedule_gtfs_dataset_key",
        "caltrans_district",
        "organization_name",
        "name",
        "sched_rt_category",
        "service_date",
    ],
)

In [13]:
# Filter for Jan and Feb
schd_vp_df2 = schd_vp_df.loc[
    (schd_vp_df.service_date == "2025-02-12")
]

In [14]:
# Drop duplicates
schd_vp_df3 = (
    schd_vp_df2[
        [
            "schedule_gtfs_dataset_key",
            "organization_name",
            "service_date",
            "sched_rt_category",
            "caltrans_district",
        ]
    ]
    .drop_duplicates(subset=["organization_name"])
    .sort_values(by=["organization_name"])
)

In [15]:
schd_vp_df3.sched_rt_category.value_counts()

schedule_and_vp    104
schedule_only       86
vp_only              2
Name: sched_rt_category, dtype: int64

## Trying to undestand why Southern California Regional Rail Authority 	 & San Bernardino County Transportation Authority 	are still "vp_only" after all the other operators have been fixed.

In [17]:
vp_only_ops = list(
    schd_vp_df3.loc[
        schd_vp_df3.sched_rt_category == "vp_only"
    ].organization_name.unique()
)

In [18]:
vp_only_ops_df = schd_vp_df.loc[schd_vp_df.organization_name.isin(vp_only_ops)]

In [26]:
vp_only_ops_sched_keys = list(vp_only_ops_df.schedule_gtfs_dataset_key.unique())

In [19]:
len(vp_only_ops)

2

In [40]:
vp_only_ops

['San Bernardino County Transportation Authority',
 'Southern California Regional Rail Authority']

In [20]:
vp_only_ops_df.groupby(["caltrans_district","organization_name", "sched_rt_category"]).agg(
    {"service_date": "max"}
)

service_date
caltrans_district          organization_name                              sched_rt_category             
07 - Los Angeles           San Bernardino County Transportation Authority schedule_only              NaT
                                                                          vp_only                    NaT
                                                                          schedule_and_vp            NaT
                           Southern California Regional Rail Authority    schedule_only       2024-06-12
                                                                          vp_only             2024-12-11
                                                                          schedule_and_vp     2024-06-12
07 - Los Angeles / Ventura San Bernardino County Transportation Authority schedule_only              NaT
                                                                          vp_only                    NaT
                                                                          schedule_and_vp            NaT
                           Southern California Regional Rail Authority    schedule_only              NaT
                                                                          vp_only             2025-02-12
                                                                          schedule_and_vp            NaT
08 - San Bernardino        San Bernardino County Transportation Authority schedule_only       2024-06-12
                                                                          vp_only             2025-02-12
                                                                          schedule_and_vp     2024-06-12
                           Southern California Regional Rail Authority    schedule_only              NaT
                                                                          vp_only                    NaT
                                                                          schedule_and_vp            NaT

## Go back to  `schedule_metrics_by_route_direction`

In [24]:
import sys

sys.path.append("../gtfs_funnel/")
import schedule_stats_by_route_direction

In [28]:
route_group_merge_cols = [
            "schedule_gtfs_dataset_key", 
            "route_id", 
            "direction_id"
        ]
        

### The two operators are already disappearing in the `trip_metrics` line 203. 

In [30]:
trip_metrics = schedule_stats_by_route_direction.assemble_scheduled_trip_metrics(
    analysis_date, GTFS_DATA_DICT
)

In [31]:
trip_metrics = trip_metrics.loc[
    trip_metrics.schedule_gtfs_dataset_key.isin(vp_only_ops_sched_keys)
]

In [33]:
vp_only_ops_sched_keys

['759ad28de7d4bb8b2bf9bb7d83655100', 'c4092405159366c705b62df938293a4e']

### Operators are already gone in `STOP_TIMES_FILE` in line 21.

In [35]:
STOP_TIMES_FILE = GTFS_DATA_DICT.rt_vs_schedule_tables.stop_times_direction

In [36]:
df = gpd.read_parquet(
        f"{RT_SCHED_GCS}{STOP_TIMES_FILE}_{analysis_date}.parquet"
    )

In [37]:
df.head(1)

,feed_key,stop_id,stop_sequence,schedule_gtfs_dataset_key,trip_instance_key,shape_array_key,stop_name,geometry,stop_meters,prior_stop_sequence,subseq_stop_sequence,stop_primary_direction,stop_pair,stop_pair_name
0,41c89143dd7679a9765226306775bb99,36498,1,0666caf3ec1ecc96b74f4477ee4bc939,00004f47348591029b238f22b41c729d,0aa3365be2ce3f9045c1369a68102f75,Downey Transit Center,POINT (172489.804 -451172.044),0.07,NaN,2.00,Unknown,36498__10457,Downey Transit Center__Firestone / Downey


In [38]:
df = df.loc[
    df.schedule_gtfs_dataset_key.isin(vp_only_ops_sched_keys)
]

In [39]:
len(df)

0

### Southern California Regional Rail Authority is vehicle positions only, sort of strange.

In [21]:
schd_vp_df3.loc[schd_vp_df3.organization_name.str.contains("Rail")]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [ ]:
schd_vp_df3.loc[schd_vp_df3.organization_name.str.contains("Metropolitan")]

In [ ]:
schd_vp_df3.loc[schd_vp_df3.organization_name.str.contains("Fleet")]

In [ ]:
schd_vp_df3.loc[schd_vp_df3.organization_name.str.contains("Ferry")]

In [ ]:
schd_vp_df3.loc[schd_vp_df3.organization_name.str.contains("Bay")]

In [ ]:
schd_vp_df3.loc[schd_vp_df3.organization_name.str.contains("Alameda")]

In [ ]:
schd_vp_df3.loc[schd_vp_df3.organization_name.str.contains("Golden")]

In [ ]:
schd_vp_df3.loc[schd_vp_df3.organization_name.str.contains("Santa Cruz")]

## Look at ferry operators and see how to incorporate them
* San Francisco Bay Area Water Emergency Transit Authority
* City of Alameda
* Golden Gate Bridge, Highway and Transportation District

### City of Alameda

In [ ]:
city_of_alameda_df = pd.read_parquet(
    schd_vp_url,
    filters=[
        [
            ("organization_name", "==", "City of Alameda"),
        ]
    ],
)

In [ ]:
city_of_alameda_df[
    [
        "route_primary_direction",
        "route_long_name",
        "route_short_name",
        "route_combined_name",
        "route_id",
        "typology",
    ]
].drop_duplicates()

In [ ]:
city_of_alameda_df.schedule_gtfs_dataset_key.unique()

In [ ]:
city_of_alameda_df.columns

#### No ferry typologies.

In [ ]:
route_typologies.loc[
    route_typologies.schedule_gtfs_dataset_key == "82f30e22dafe8156367297eb9a316c57"
]

### San Francisco Bay Area Water Emergency Transit Authority
* Duplicates City of Alameda data except for Oyster Bay.

In [ ]:
weta_df = pd.read_parquet(
    schd_vp_url,
    filters=[
        [
            (
                "organization_name",
                "==",
                "San Francisco Bay Area Water Emergency Transit Authority",
            ),
        ]
    ],
)

In [ ]:
weta_df[
    [
        "route_primary_direction",
        "route_long_name",
        "route_short_name",
        "route_combined_name",
        "route_id",
        "typology",
    ]
].drop_duplicates()

### Golden Gate
* Only Bus Routes.
* This should be schedule too? 

In [ ]:
goldengate_df = pd.read_parquet(
    schd_vp_url,
    filters=[
        [
            (
                "organization_name",
                "==",
                "Golden Gate Bridge, Highway and Transportation District",
            ),
        ]
    ],
)

In [ ]:
goldengate_df.sched_rt_category.value_counts()

In [ ]:
goldengate_df.columns

In [ ]:
goldengate_df.schedule_gtfs_dataset_key.unique()

In [ ]:
goldengate_df[
    [
        "route_primary_direction",
        "route_long_name",
        "route_short_name",
        "route_combined_name",
        "route_id",
        "typology",
    ]
].drop_duplicates()

## Go back to schedule portion of `merge_data` and see why these vp_only operators are being dropped.

In [ ]:
sched_data = merge_data.concatenate_schedule_by_route_direction(analysis_date_list)

In [ ]:
sched_data.head(1)

In [ ]:
schd_vp_df2.columns

In [ ]:
vp_only_sched_keys = list(vp_only_ops_df.schedule_gtfs_dataset_key.unique())

In [ ]:
sched_data2 = sched_data.loc[
    sched_data.schedule_gtfs_dataset_key.isin(vp_only_sched_keys)
]

In [ ]:
sched_data2

### Digging into `gtfs_funnel/schedule_stats_by_route_direction.py`

In [ ]:
import sys

sys.path.append("../gtfs_funnel/")
import schedule_stats_by_route_direction

#### Line 203

In [ ]:
trip_metrics = schedule_stats_by_route_direction.assemble_scheduled_trip_metrics(
    analysis_date, GTFS_DATA_DICT
)

In [ ]:
trip_metrics = trip_metrics.loc[
    trip_metrics.schedule_gtfs_dataset_key.isin(vp_only_sched_keys)
]

In [ ]:
trip_metrics.head(1)

In [ ]:
len(vp_only_sched_keys)

#### Some operators havae duplicative schedule_gtfs_dataset_keys

In [ ]:
trip_sched_keys = set(list(trip_metrics.schedule_gtfs_dataset_key.unique()))

In [ ]:
type(trip_sched_keys)

In [ ]:
og = set(vp_only_sched_keys)

In [ ]:
type(og)

In [ ]:
og - trip_sched_keys

#### Some `schedule_gtfs_datset_keys` are repeated for two different organizations
* SF Bay Area WETA = has two different keys
* SoCal Regional Rail Authority = also has two keys
* City of Alameda & SF WETA share the same key. 
* San Bernardino & Southern California Regional Rail share the same key.
* Maybe need to drop duplicates by schedule_gtfs_dataset_key as well. 

In [ ]:
vp_only_ops_df[["schedule_gtfs_dataset_key", "organization_name"]].drop_duplicates()

In [ ]:
trip_metrics.direction_id = trip_metrics.direction_id.fillna(0)

In [ ]:
len(trip_metrics)

### Why doesn't anything show up with this function `schedule_metrics_by_route_direction`

In [ ]:
route_dir_metrics = (
    schedule_stats_by_route_direction.schedule_metrics_by_route_direction(
        trip_metrics, analysis_date, route_group_merge_cols
    )
)

In [ ]:
route_dir_metrics.head(1)

#### Line 148

In [ ]:
group_merge_cols = ["schedule_gtfs_dataset_key", "route_id", "direction_id"]

In [ ]:
service_freq_df = gtfs_schedule_wrangling.aggregate_time_of_day_to_peak_offpeak(
        trip_metrics, group_merge_cols, long_or_wide="long"
    )

In [ ]:
service_freq_df.schedule_gtfs_dataset_key.unique()

In [ ]:
metrics_df = (
        trip_metrics.groupby(group_merge_cols, observed=True, group_keys=False, dropna=False)
        .agg(
            {
                "median_stop_meters": "mean",
                # take mean of the median stop spacing for trip
                # does this make sense?
                # median is the single boiled down metric at the trip-level
                "scheduled_service_minutes": "mean",
            }
        )
        .reset_index()
        .rename(
            columns={
                "median_stop_meters": "avg_stop_meters",
                "scheduled_service_minutes": "avg_scheduled_service_minutes",
            }
        )
    )

In [ ]:
metrics_df.schedule_gtfs_dataset_key.unique()

In [ ]:
from shared_utils.rt_utils import METERS_PER_MILE

In [ ]:
metrics_df = metrics_df.assign(
        avg_stop_miles=metrics_df.avg_stop_meters.divide(METERS_PER_MILE).round(2)
    ).drop(columns=["avg_stop_meters"])

round_me = ["avg_stop_miles", "avg_scheduled_service_minutes"]
metrics_df[round_me] = metrics_df[round_me].round(2)

#### Line 179 = where the routes of interest are getting deleted.
* Delete out `.pipe(helpers.remove_shapes_outside_ca)` to bring back in ferry operators.
* I think the routes on the water are considered out of bounds with the California geography.

In [ ]:
common_shape = gtfs_schedule_wrangling.most_common_shape_by_route_direction(
        analysis_date
    ).pipe(helpers.remove_shapes_outside_ca)


In [ ]:
common_shape = common_shape.loc[
    common_shape.schedule_gtfs_dataset_key.isin(vp_only_sched_keys)
]

In [ ]:
common_shape.schedule_gtfs_dataset_key.nunique()

In [ ]:
common_shape2 = gtfs_schedule_wrangling.most_common_shape_by_route_direction(
        analysis_date
    )

In [ ]:
common_shape2 = common_shape2.loc[
    common_shape2.schedule_gtfs_dataset_key.isin(vp_only_sched_keys)
]

In [ ]:
len(common_shape2)

In [ ]:
common_shape2.explore("schedule_gtfs_dataset_key")

## Seeing which graphs are vp_only using 

In [ ]:
stop

In [ ]:
import _report_utils
import altair as alt
import yaml

In [ ]:
with open("readable.yml") as f:
    readable_dict = yaml.safe_load(f)

In [ ]:
with open("color_palettes.yml") as f:
    color_dict = yaml.safe_load(f)

In [ ]:
df = weta_df.copy()

In [ ]:
# Round float columns
float_columns = df.select_dtypes(include=["float"])
for i in float_columns:
    df[i] = df[i].round(2)

# Multiply percent columns to 100%
pct_cols = df.columns[df.columns.str.contains("pct")].tolist()
for i in pct_cols:
    df[i] = df[i] * 100

In [ ]:
# Add column to create rulers for the charts
df["ruler_100_pct"] = 100
df["ruler_for_vp_per_min"] = 2

# Add a column that flips frequency to be every X minutes instead
# of every hour.
df["headway_in_minutes"] = 60 / df.frequency

In [ ]:
df.route_primary_direction = df.route_primary_direction.fillna("None")

In [ ]:
df = _report_utils.replace_column_names(df)

In [ ]:
routes_list = df["Route"].unique().tolist()

route_dropdown = alt.binding_select(
    options=routes_list,
    name="Routes: ",
)
# Column that controls the bar charts
xcol_param = alt.selection_point(
    fields=["Route"], value=routes_list[0], bind=route_dropdown
)

# Filter for only rows that are "all day" statistics
all_day = df.loc[df["Period"] == "all_day"].reset_index(drop=True)

In [ ]:
timeliness_df = section2.timeliness_trips(df)

In [ ]:
timeliness_df.head(2)

In [ ]:
def pct_vp_journey(df: pd.DataFrame, col1: str, col2: str) -> pd.DataFrame:
    """
    Reshape the data for the charts that display the % of
    a journey that recorded 2+ vehicle positions/minute.
    """
    to_keep = [
        "Date",
        "Organization",
        "dir_0_1",
        col1,
        col2,
        "Route",
        "Period",
        "ruler_100_pct",
    ]
    df2 = df[to_keep]

    df3 = df2.melt(
        id_vars=[
            "Date",
            "Organization",
            "Route",
            "dir_0_1",
            "Period",
            "ruler_100_pct",
        ],
        value_vars=[col1, col2],
    )

    df3 = df3.rename(
        columns={"variable": "Category", "value": "% of Actual Trip Minutes"}
    )
    return df3

In [ ]:
sched_journey_vp = pct_vp_journey(
    all_day,
    "% Scheduled Trip w/ 1+ VP/Minute",
    "% Scheduled Trip w/ 2+ VP/Minute",
)

In [ ]:
sched_journey_vp.head(2)

In [ ]:
route_stats_df = section2.route_stats(df)

In [ ]:
route_stats_df.head(2)

## Build this into a function

In [ ]:
def load_vp_metrics(organization: str) -> pd.DataFrame:
    """
    Load schedule versus realtime file.
    """
    schd_vp_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.route_schedule_vp}.parquet"

    # Keep only rows that are found in both schedule and real time data
    df = pd.read_parquet(
        schd_vp_url,
        filters=[
            [
                ("organization_name", "==", organization),
            ]
        ],
    )

    # Delete duplicates
    df = df.drop_duplicates().reset_index(drop=True)

    # Round float columns
    float_columns = df.select_dtypes(include=["float"])
    for i in float_columns:
        df[i] = df[i].round(2)

    # Multiply percent columns to 100%
    pct_cols = df.columns[df.columns.str.contains("pct")].tolist()
    for i in pct_cols:
        df[i] = df[i] * 100

    # Add column to create rulers for the charts
    df["ruler_100_pct"] = 100
    df["ruler_for_vp_per_min"] = 2

    # Add a column that flips frequency to be every X minutes instead
    # of every hour.
    df["headway_in_minutes"] = 60 / df.frequency

    # Replace missing values in route_primary_direction
    df.route_primary_direction = df.route_primary_direction.fillna(df.direction_id)

    # Replace column names
    df = _report_utils.replace_column_names(df)

    return df

In [ ]:
dumbardton_df = load_vp_metrics("Dumbarton Bridge Regional Operations Consortium")

In [ ]:
dumbardton_df.head(1)

In [ ]:
socal_rail_df = load_vp_metrics("Southern California Regional Rail Authority")

In [ ]:
gg_df = load_vp_metrics("Golden Gate Bridge, Highway and Transportation District")

In [ ]:
def filtered_route(
    df: pd.DataFrame,
) -> alt.Chart:
    """
    This combines all the charts together, controlled by a single
    dropdown.

    Resources:
        https://stackoverflow.com/questions/58919888/multiple-selections-in-altair
    """
    # Create dropdown
    routes_list = df["Route"].unique().tolist()

    route_dropdown = alt.binding_select(
        options=routes_list,
        name="Routes: ",
    )
    # Column that controls the bar charts
    xcol_param = alt.selection_point(
        fields=["Route"], value=routes_list[0], bind=route_dropdown
    )

    # Filter for only rows that are "all day" statistics
    all_day = df.loc[df["Period"] == "all_day"].reset_index(drop=True)

    # Manipulate the df for some of the metrics
    timeliness_df = section2.timeliness_trips(df)
    sched_journey_vp = section2.pct_vp_journey(
        all_day,
        "% Scheduled Trip w/ 1+ VP/Minute",
        "% Scheduled Trip w/ 2+ VP/Minute",
    )
    route_stats_df = section2.route_stats(df)

    # Create the charts
    timeliness_trips_dir_0 = (
        (
            section2.base_facet_chart(
                timeliness_df.loc[timeliness_df["dir_0_1"] == 0],
                0,
                "value",
                "variable",
                "Period",
                readable_dict["timeliness_trips_graph"]["title"],
                readable_dict["timeliness_trips_graph"]["subtitle"],
                color_dict["tri_color2"],
            )
        )
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )
    timeliness_trips_dir_1 = (
        (
            section2.base_facet_chart(
                timeliness_df.loc[timeliness_df["dir_0_1"] == 1],
                1,
                "value",
                "variable",
                "Period",
                readable_dict["timeliness_trips_graph"]["title"],
                "",
                color_dict["tri_color2"],
            )
        )
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )

    speed_graph_dir_0 = (
        section2.grouped_bar_chart(
            df.loc[df.dir_0_1 == 0],
            "Period",
            "Speed (MPH)",
            "Period",
            readable_dict["speed_graph_dir_0"]["title"],
            readable_dict["speed_graph_dir_0"]["subtitle"],
            color_dict["tri_color2"],
        )
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )
    speed_graph_dir_1 = (
        section2.grouped_bar_chart(
            df.loc[df.dir_0_1 == 1],
            "Period",
            "Speed (MPH)",
            "Period",
            readable_dict["speed_graph_dir_1"]["title"],
            readable_dict["speed_graph_dir_0"]["subtitle"],
            color_dict["tri_color2"],
        )
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )
    vp_per_min_graph = (
        (
            section2.base_facet_with_ruler_chart(
                all_day,
                "Average VP per Minute",
                "ruler_for_vp_per_min",
                readable_dict["vp_per_min_graph"]["title"],
                readable_dict["vp_per_min_graph"]["subtitle"],
                color_dict["vp_domain"],
                color_dict["vp_range"],
            )
        )
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )

    sched_vp_per_min = (
        section2.base_facet_circle(
            sched_journey_vp,
            "% of Actual Trip Minutes",
            "Category",
            "ruler_100_pct",
            readable_dict["sched_vp_per_min_graph"]["title"],
            readable_dict["sched_vp_per_min_graph"]["subtitle"],
            color_dict["tri_color2"],
        )
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )
    spatial_accuracy = (
        section2.base_facet_with_ruler_chart(
            all_day,
            "% VP within Scheduled Shape",
            "ruler_100_pct",
            readable_dict["spatial_accuracy_graph"]["title"],
            readable_dict["spatial_accuracy_graph"]["subtitle"],
            color_dict["spatial_accuracy_domain"],
            color_dict["spatial_accuracy_range"],
        )
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )
    # Separate out the charts themetically.
    ride_quality = section2.divider_chart(
        df, readable_dict["ride_quality_graph"]["title"]
    )
    data_quality = section2.divider_chart(
        df, readable_dict["data_quality_graph"]["title"]
    )

    # Combine all the charts
    chart_list = [
        ride_quality,
        timeliness_trips_dir_0,
        timeliness_trips_dir_1,
        speed_graph_dir_0,
        speed_graph_dir_1,
        data_quality,
        vp_per_min_graph,
        sched_vp_per_min,
        spatial_accuracy,
    ]

    chart = alt.vconcat(*chart_list)

    return chart

In [ ]:
filtered_route(gg_df)

In [ ]:
filtered_route(socal_rail_df)

In [ ]:
filtered_route(dumbardton_df)

### `Average Scheduled Minutes` chart doesn't work.

In [ ]:
(
    (
        section2.base_facet_chart(
            timeliness_df.loc[timeliness_df["dir_0_1"] == 1],
            1,
            "value",
            "variable",
            "Period",
            readable_dict["timeliness_trips_graph"]["title"],
            "",
        )
    )
    .add_params(xcol_param)
    .transform_filter(xcol_param)
)

In [ ]:
df.headway_in_minutes = df.headway_in_minutes.fillna(0)

### `Frequency` doesn't work.

In [ ]:
(
    section2.frequency_chart(
        df,
        0,
        readable_dict["frequency_graph"]["title"],
        readable_dict["frequency_graph"]["subtitle"],
    )
    .add_params(xcol_param)
    .transform_filter(xcol_param)
)

#### `speed` also doesn't work.

In [ ]:
(
    section2.grouped_bar_chart(
        df.loc[df.dir_0_1 == 0],
        "Period",
        "Speed (MPH)",
        "Period",
        readable_dict["speed_graph_dir_0"]["title"],
        readable_dict["speed_graph_dir_0"]["subtitle"],
    )
    .add_params(xcol_param)
    .transform_filter(xcol_param)
)

In [ ]:
all_day.head(1).T

In [ ]:
(
    (
        section2.base_facet_with_ruler_chart(
            all_day.loc[all_day.dir_0_1 == 0],
            "Average VP per Minute",
            "ruler_for_vp_per_min",
            readable_dict["vp_per_min_graph"]["title"],
            readable_dict["vp_per_min_graph"]["subtitle"],
            color_dict["vp_domain"],
            color_dict["vp_range"],
        )
    )
    .add_params(xcol_param)
    .transform_filter(xcol_param)
)

In [ ]:
(
    (
        section2.base_facet_with_ruler_chart(
            all_day.loc[all_day.dir_0_1 == 1],
            "Average VP per Minute",
            "ruler_for_vp_per_min",
            readable_dict["vp_per_min_graph"]["title"],
            readable_dict["vp_per_min_graph"]["subtitle"],
            color_dict["vp_domain"],
            color_dict["vp_range"],
        )
    )
    .add_params(xcol_param)
    .transform_filter(xcol_param)
)

In [ ]:
sched_journey_vp.columns

In [ ]:
sched_journey_vp = sched_journey_vp.rename(columns={"dir_0_1": "Direction"})

In [ ]:
(
    section2.base_facet_circle(
        sched_journey_vp,
        "% of Actual Trip Minutes",
        "Category",
        "ruler_100_pct",
        readable_dict["sched_vp_per_min_graph"]["title"],
        readable_dict["sched_vp_per_min_graph"]["subtitle"],
    )
    .add_params(xcol_param)
    .transform_filter(xcol_param)
)

#### The bars are stacked because the direction 0/1 are coded as "None" in `route_primary_direction`
* Need to drop Direction and rename `dir_0_1` as Direction.

In [ ]:
all_day = all_day.drop(columns=["Direction"]).rename(columns={"dir_0_1": "Direction"})

In [ ]:
(
    section2.base_facet_with_ruler_chart(
        all_day,
        "% VP within Scheduled Shape",
        "ruler_100_pct",
        readable_dict["spatial_accuracy_graph"]["title"],
        readable_dict["spatial_accuracy_graph"]["subtitle"],
        color_dict["spatial_accuracy_domain"],
        color_dict["spatial_accuracy_range"],
    )
    .add_params(xcol_param)
    .transform_filter(xcol_param)
)

## Why is <i>Golden Gate Bridge, Highway and Transportation District</i> `vp_only`? It should have schedule data!

In [ ]:
import merge_data

In [ ]:
analysis_date

In [ ]:
sched_df = merge_data.concatenate_schedule_by_route_direction(analysis_date_list)

In [ ]:
sched_df.loc[sched_df.schedule_gtfs_dataset_key == "aea4108997c66a74fbdae27b34b69fde"]